# **Finding Lane Lines on the Road**  
  
One of the most basic aspects of driving a car (safely) is to make sure the car is within a lane. We as a human driver do it almost instictively or say involuntarily by constantly correcting to make sure the vehicle doesn't veer off into another vehicles path. What we humans do visually can be replicated in an autonomous vehicle using a series of Computer Vision techniques. The objective of this implementation and the process pipeline required to achieve this is explained below.  
  
## Objective  
  
The objective of this project is to implement a set of functions which combine together to identify lane lines on a road. When presented with a video stream from a front view camera, this implementation should identify the left and right lane markings and highlight them as a thick red line extending from the front of the vehicle into the plane of the image away from it as shown below.  
  
<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>
  
## Workspace  
  
For this nanodegree, while working on python based projects, I've decided to use Google Colab as my workspace. Gogogle Colab really freed me to go to the hastle of installing the tool chain which often gives installation nightmare. Few of the advantages of using Colab I have mentioned below.

1. Most python libraries required for this nano degree are readily available on Colab and adding new libraries is very easy as well  
2. Gives me flexibility of carrying my workspace whereever I want  
3. Can easily fire up GPU and TPU supported instances and speed up some of the training process  
4. Can mount my google Drive on Colab and have a copy of my files on the cloud 
5. Can commit my work to github directly from colab  
  
## Lane Detection Pipeline  
The input for the lane detection task , as mentioned earlier, is a video stream from the front view camera. The video stream in reality is only a continuous stream of individual image frames. So, our lane detection effectively starts by extracting each frame from a video. Once we have the individual frame, the processing pipeline follows  
  
1. Convert the color image to gray scale image 
2. Applying Gaussian blur  
3. Detect edges in the image using Canny Edge Detector  
4. Mask and retain only the region of interest from the entire image  
5. Apply Hough Transform on the masked edge features to identify lines  
6. Identify the left and right lane lines from their slopes  
7. From all the detected left and right lanes, get the average left line's slope and intercept and the right line's slope and intercept  
8. Use the average slope to draw an extrapolated line in the region of interest 
9. Overlay the extrapolated line on the original image  
10. Run the function on the video which is nothing but the set of images


### Orginial images ### 

<figure>
 <img src="test_images_output/Original.png" width="380" alt="Example Image" />
 <figcaption>
 <p style="text-align: center;"> Orginal example image having solid white curve on the right and dotted white curve on the left. </p> 
 </figcaption>
</figure>


### Grayscale and Blurring ### 
The basic idea behind lane line detection is the identification of the edges that constitute the lane. Although edge detection can be done on a RGB image, the effective edge is determined by the channel with the maximum gradient. So, for efficient edge detection, the first task is to convert to **gray** scale image. Converting the gray scale makes the image into a 8 bit image which can take 256 value (0, 255) -- **0 (black), 255(white)**
 
 <figure>
 <img src="test_images_output/Gray.png" width="380" alt="Gray Image" />
 <figcaption>
 <p style="text-align: center;"> Converting to Gray scale. </p> 
 </figcaption>
</figure>

 <p></p>
Blurring is another important technique which helps reduce high frequency content in the image and hence reduces noise and helps in better edge extraction. There are several blurring methods available in openCV. The most commonly used method is the Gaussian Blur. In this method, a ** M x N ** kernel is convolved with the input image and the intensity of each pixel is modified to the weighted average of the intensities of the pixels in the kernel. 

 <figure>
 <img src="test_images_output/Gaussian.png" width="380" alt="Gaussian blur Image" />
 <figcaption>
 <p style="text-align: center;"> Applying Gaussian Blur. </p> 
 </figcaption>
</figure>



### Canny Edge Detection  
Canny edge detection is a technique used to identify gradients of any orientation in an image that helps us extract the structural information in an image. 

- First the Canny edge algorithm does the Gaussian Smoothing, which is to suppress noise and spurious gradients by averaging.
- Second computes to gradient and detects the strong edge.
- It is recommended to keep the ratio of low_threshold to high threshold to 1:2 or 1:3 . Algorthim detects the strong edge pixels above the high_threshold and rejects pixels below the low_threshold. pixels in between are included as long as they are connected to strong edges. Output edges is a binary image with white pixels tracing out the detected edges and black pixels everywhere else.

 <figure>
 <img src="test_images_output/Canny_edge.png" width="380" alt="Canny edge Image" />
 <figcaption>
 <p style="text-align: center;"> Compute to gradient by applying Canny edge. </p> 
 </figcaption>
</figure>

 
 ### Region of Interest
  
 Since we are only interested in the lane lines, the region of the image that is not immediately in front of the vehicle, like the cars in the adjascent lane, the barriers at the extreme left and right can all be neglected. This is achieved by masking the blurred image with a polygon that covers only the region of interest. This polygon is defined by four vertices [bottom_left, top_left, top_right, bottom_right]. Once we define the vertices, we mask the area of interest to suppress the noise by blanking areas outside region of interest.
 
 
 <figure>
 <img src="test_images_output/ROI_masked.png" width="380" alt="Maksed ROI" />
 <figcaption>
 <p style="text-align: center;"> Masking the region of interest. </p> 
 </figcaption>
</figure>
 
 Two different polygons were used for the two different sets of videos in the project. The solidWhiteRIght and the solidYellowLeft videos have the same mask where as the challenge video has a different mask. 
    
### Hough Transform

Given a set of edge points in a binary image, Hough transformation is used to find a line that passes through all these points. 
A line in a cartesian space is represented by its slope $m$ and intercept $b$ as 
$y=mx+b$

The same line when transformed in the $m-b$ space will be a point as shown in the image below

 <figure>
 <img src="test_images_output/hough-cartesian.png" width="380" alt="Hough Vs Cartesian" />
 <figcaption>
 <p style="text-align: center;"> Line in a image space will be a point in the Hough space. </p> 
 </figcaption>
</figure>

Where this representation fails is when the line is vertical with an infinite slope. To over come this, the $\rho-\theta$ space is used. The same line in $\rho-\theta$  space can be represented as 
$\rho = x*cos(\theta) + y*sin(\theta)$

Curves generated by collinear points in the image space intersect at common $(\rho,\theta)$ in the Hough transform space. The more curves intersect at a point, the more vote the line gets. Thus by using a **threshold** number of votes per line, we can select prominent lines in an image. 

 <figure>
 <img src="test_images_output/hough-thetha.png" width="380" alt="Hough Vs Cartesian Polar" />
 <figcaption>
 <p style="text-align: center;"> Line in a image space's polar coordinate will be a point in the Hough space. </p> 
 </figcaption>
</figure>

### Identify the left and right line and get the average slope

Once the lines are obtained from the Hough transform, we can pass the line end points to a first order polynomial fit function in numpy to get the slope and y-intercept of the fitted line. We can identify the left and right line by checking their slopes. The line with a negative slope is the left line and the one with the positive slope is the right line. This is due to the fact that for an image the (0,0) is at the top left corner of the image. 

The average slope of these line making up the left and right lines can be obtained by taking a weighted average of all the individual line slopes. The slopes are weighted based on the length of the line and hence the average slope is influenced more by the longer lines than the shorter ones. 

 <figure>
 <img src="test_images_output/Hough.png" width="380" alt="Hough" />
 <figcaption>
 <p style="text-align: center;"> After performing Hough transform. </p> 
 </figcaption>
</figure>


### Draw and overlay the extrapolated lane lines on the image

Using the averaged line slopes and intercepts , we can find the x co-ordinate of the start and end of the line. These lines are drawn on the original image with a set transparency to highlight the identified lane lines as shown below

 <figure>
 <img src="test_images_output/Final.png" width="380" alt="Final" />
 <figcaption>
 <p style="text-align: center;"> After overlaying the image to the orginal image.</p> 
 </figcaption>
</figure>
    
## Limitations 
    
There are several limitations to this implementation. To list a few
    
1. Even with the simpler videos, the lane line flickers when there is some noise in the image. This stems from the fact that the average slope of the lane line is computed fresh for every frame and if edges with different slopes are detected between consecutive frames, the lane line is redrawn and appears as a flicker. This can be eliminated by keeping a history of the slopes of the previous few frames and taking an average of that would result in a more stable lane line. 
2. In the challenge video, when the roads switches from asphalt to concrete, the intersection is identified as an edge. To avoid this, in the current implementation every edge is checked for its angle and if the edge is less than $25^0$ it is not drawn. As a result the portion of the video that switches from asphalt to concrete does not have the lane marking. Even after thresholding the grayscale image, the problem persisted. 
3. All lane lines are drawn as straignt lines. Hence the curvature of the road is not captured. A polynomial curve fitting needs to be implemented to be able to draw curved lane marking. 